# Mission to Mars: Scraping & Analysis

In [2]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

### News Scraping

In [3]:
def init_browser():
    browser = Browser("chrome", headless=False)
    return browser

In [4]:
def scrape():
    browser = init_browser()
    url_news = "https://mars.nasa.gov/news/"
    browser.visit(url_news)
    time.sleep(1)
    mars_news = {}
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    mars_news["news_title"] = soup.find("div", class_ = "content_title").next_element.get_text()
    mars_news["news_para"] = soup.find("div", class_ = "article_teaser_body").get_text()
    return mars_news

In [5]:
scrape()

{'news_para': 'A post-test inspection of the composite structure for a heat shield to be used on the Mars 2020 mission revealed that a fracture occurred during structural testing.',
 'news_title': 'Results of Heat Shield Testing'}

In [6]:
mars = scrape()
print("------------------")
print(mars["news_title"])
print(mars["news_para"])

------------------
Results of Heat Shield Testing
A post-test inspection of the composite structure for a heat shield to be used on the Mars 2020 mission revealed that a fracture occurred during structural testing.


In [7]:
def scrape_img():
    browser = init_browser()
    url = "http://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    image = soup.find("a", class_ = "button fancybox")
    featured_image_url = "https://www.jpl.nasa.gov" + image["data-fancybox-href"]
    
    return featured_image_url

feature_image_url = scrape_img()
print(feature_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17896_ip.jpg


In [8]:
def weather_scrape():
    browser = init_browser()
    url = "https://twitter.com/marswxreport?leng=en"
    
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    weather = soup.find_all('p', class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
    
    for i in weather:
        j = i.text.split()
        if j[0] != "Sol":
            continue
        else:
            mars_weather = i.text
            break
            
    return mars_weather

mars_weather = weather_scrape
print(mars_weather)

<function weather_scrape at 0x110a19730>


In [9]:
def fact_scrape():
    url = "https://space-facts.com/mars/"
    
    mars_table = pd.read_html(url)
    
    return mars_table

In [10]:
table = fact_scrape()

mars_fact_df = table[0]
mars_fact_df.columns = ["Characteristic", "Measurement"]

mars_fact_df.head()

,Characteristic,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [17]:
browser = init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_url = "https://astrogeology.usgs.gov"

browser.visit(url)
time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, 'lxml')

url_list = []
title_list =[]
jpg_url_list =[]
hemisphere_image_urls = []

results = soup.find('div', class_="collapsible results")
links = results.find_all('a', class_="item product-item")
titles= results.find_all('h3')

for link in links:
    url_list.append(base_url + link["href"])
    
for title in titles:
    title_list.append(title.text)
    
for item in url_list:
    browser.visit(item)
    time.sleep(1)
    
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    
    downloads = soup.find('div', class_="downloads")
    jpg = downloads.find_all('a')
    jpg_url = jpg[0]["href"]
    jpg_url_list.append(jpg_url)
    

for i in range(0,len(title_list)):
    hemisphere_image_urls.append({"title": title_list[i], "img_url": jpg_url_list[i]})

IndexError: list index out of range

In [18]:
scraped_data= {}

scraped_data["news"] = mars
scraped_data["feat_img"] = featured_image_url
scraped_data["weather"] = mars_weather
scraped_data["facts"] = table
scraped_data["hemispheres"] = hemisphere_image_urls

NameError: name 'featured_image_url' is not defined

In [15]:
browser = init_browser()
url_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


browser.visit(url_hemi)
soup = BeautifulSoup(browser.html, 'html.parser')


headers = []
titles = soup.find_all('h3')

for title in titles:
    headers.append(title.text)

images = []
counter = 0
for thumb in headers:
    browser.find_by_css('img.thumb')[counter].click()
    images.append(browser.find_by_text('Sample')['href'])
    browser.back()
    counter = counter + 1
    
hemisphere_image_urls = []
count = 0
for item in images:
    hemisphere_image_urls.append({"title": headers[counter], "img_url": images[counter]})
    counter + 1

print(hemisphere_image_urls)

IndexError: list index out of range